<a href="https://colab.research.google.com/github/liuchiente/UDICatNCHU/blob/main/BERT_UDICatNCHU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

掛載目錄和基本設定

In [ ]:
#掛載Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#切換預設目錄
import os
SYS_DIR = "/content/drive/My Drive/Colab Notebooks/UDICatAlBert"
if not os.path.isdir(SYS_DIR):
    os.mkdir(SYS_DIR)
os.chdir(SYS_DIR)

In [ ]:
#安裝Pytorch
!pip install torch torchvision

In [ ]:
#引入Pytorch
import torch
print(torch.__version__)
torch.cuda.is_available()

1.8.0+cu101


True

安裝Bert

In [ ]:
#安裝Bert
!pip install transformers

In [ ]:
!pip install sentencepiece 

In [ ]:
#設定使用GPU做訓練
isdevice = torch.cuda.is_available()
if not isdevice:
    print('CUDA is not available.')
else:
    print('CUDA is available!')

CUDA is available!


In [ ]:
#選擇Bert版本
%tensorflow_version 2.x

In [ ]:
#使用Bert
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

下載Kaggle資料集

In [ ]:
#安裝Kaggle,後續用來下載資料集,只要下載一次就好
!pip install kaggle

In [ ]:
{"username":"jjdliu","key":"b28c30316bfb0f90927832ff6ed95c31"}

In [ ]:
api_token = {"username":"jjdliu","key":"b28c30316bfb0f90927832ff6ed95c31"}
import json
import zipfile
import os
 
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists(SYS_DIR):
    os.makedirs(SYS_DIR)
os.chdir(SYS_DIR)
#!kaggle datasets download -d sentiment-analysis-on-movie-reviews --force
!kaggle competitions download -c sentiment-analysis-on-movie-reviews
!ls /kaggle

  0% 0.00/1.28M [00:00<?, ?B/s]
100% 1.28M/1.28M [00:00<00:00, 41.8MB/s]
  0% 0.00/583k [00:00<?, ?B/s]
100% 583k/583k [00:00<00:00, 81.0MB/s]
  0% 0.00/494k [00:00<?, ?B/s]
100% 494k/494k [00:00<00:00, 68.1MB/s]
dogs-cats-images.zip


In [ ]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


資料前處理

In [ ]:
#引入函式
import numpy as np
import pandas as pd

In [ ]:
#解壓縮資料集
!unzip train.tsv.zip

Archive:  train.tsv.zip
replace train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ｎ
error:  invalid response [Ｎ]
replace train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
#載入資料
train=pd.read_csv('train.tsv.zip',encoding = "ISO-8859-1",delimiter="\t")
submit=pd.read_csv('sampleSubmission.csv',encoding = "ISO-8859-1")
test=pd.read_csv('test.tsv.zip',encoding = "ISO-8859-1",delimiter="\t")

In [ ]:
#切分出Train和Valid的CSV,Train當訓練資料,Valid驗證效果
def split_csv(infile, trainfile, valtestfile, seed=999, ratio=0.2):
    df = pd.read_csv(infile,encoding = "ISO-8859-1",delimiter="\t")
    idxs = np.arange(df.shape[0])	
    np.random.seed(seed)	
    np.random.shuffle(idxs)	
    val_size = int(len(idxs) * ratio)	
    #index=False 不存索引, True 儲存原始索引
    df.iloc[idxs[:val_size], :].to_csv(valtestfile,index=True)	
    df.iloc[idxs[val_size:], :].to_csv(trainfile,index=True)

In [ ]:
#切出8-2比例
split_csv(infile=SYS_DIR+'/train.tsv.zip',trainfile=SYS_DIR+'/b_train.csv',valtestfile=SYS_DIR +'/b_valid.csv',seed=999,ratio=0.2)

In [ ]:
#重新讀入切分好的2個資料集
train_b=pd.read_csv('b_train.csv',encoding = "ISO-8859-1",delimiter=",",index_col=0)
valid_b=pd.read_csv('b_valid.csv',encoding = "ISO-8859-1",delimiter=",",index_col=0)

In [ ]:
#轉資料集為特徵
def convert_csv_to_feature(pds):
  tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
  #最大字數
  max_len=0
  token_pics=[]
  token_ids=[]
  labels=[]
  for index, row in pds.iterrows():
    token_pic=tokenizer.tokenize(row['Phrase'])
    token_id=tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(token_pic))
    if len(token_id)>max_len:
      max_len=len(token_id)
    token_pics.append(token_pic)
    token_ids.append(token_id)
    labels.append(row['Sentiment'])

  #是否大於512
  assert max_len<=512
  attension_ids=[]
  for index in range(len(token_ids)):
    token_id=token_ids[index]
    org_len=len(token_id)
    #id補齊512個位數
    for i in range(org_len,max_len):
      token_id.append(0)
    token_ids[index]=token_id
    #建立attension,有句子地方為1,無句子地方為0
    attension_id=[]
    for i in range(0,org_len):
      attension_id.append(1)
    for i in range(org_len,max_len):
      attension_id.append(0)
    attension_ids.append(attension_id) 

  token_type_ids=[[0]*max_len for i in range(len(pds['Phrase']))]
  data_features={'input_ids':token_ids,
               'token_type_ids':token_type_ids,
               'attension_mask':attension_ids,
               'labels':labels}
  return data_features

In [ ]:
#得到訓練和驗證用的特徵
train_data_features=convert_csv_to_feature(train_b)
valid_data_features=convert_csv_to_feature(valid_b)

In [ ]:
#確認特徵編碼是否正確
print(train_data_features['input_ids'][5999])
print(tokenizer.convert_ids_to_tokens(train_data_features['input_ids'][5999]))
print(train_data_features['token_type_ids'][5999])
print(train_data_features['attension_mask'][5999])
print(train_data_features['labels'][5999])

In [ ]:
#產生DataSet
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    token_type_ids = data_feature['token_type_ids']
    attention_mask = data_feature['attension_mask']
    labels = data_feature['labels']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_token_type_ids = torch.tensor([token_type_id for token_type_id in token_type_ids], dtype=torch.long)
    all_attention_mask_ids = torch.tensor([attention_id for attention_id in attention_mask], dtype=torch.long)
    all_labels = torch.tensor([label for label in labels], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_token_type_ids, all_attention_mask_ids, all_labels)

    return dataset

In [ ]:
train_o_dataset=makeDataset(train_o_data_features)

In [ ]:
#轉資料集
train_dataset=makeDataset(train_data_features)
valid_dataset=makeDataset(valid_data_features)

In [ ]:
#得標籤數,Model設定參數時會用到
label_num = len(train_b.groupby(['Sentiment']).nunique())
print(label_num)

5


In [ ]:
# 計算正確值
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

訓練和驗證

In [ ]:
# set device
device = torch.device('cuda')

train_dataloader = DataLoader(train_dataset ,batch_size=4 ,shuffle=True)
test_dataloader = DataLoader(valid_dataset ,batch_size=4 ,shuffle=True)

#用英文Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=label_num)
model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(optimizer_grouped_parameters, lr=Learning_rate, eps=1e-8)

for epoch in range(5):
    # 訓練模式
    model.train()
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    for batch_dict in train_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],             
            attention_mask = batch_dict[2],             
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]
            
        train_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_train_correct += train_correct
        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    Average_train_correct = round(All_train_correct/count, 3)
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],            
            attention_mask = batch_dict[2],           
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        test_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_test_correct += test_correct
        AllTestLoss += loss.item()

        count += 1
        
    Average_test_correct = round(All_test_correct/count, 3)
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + ' 正確率為' + str(Average_train_correct)+ '，測試模式，loss為:' + str(Average_test_loss) + ' 正確率為' + str(Average_test_correct))
    
# 模型存檔
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained('trained_model')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

第1次訓練模式，loss為:0.799 正確率為66.713，測試模式，loss為:0.718 正確率為70.197
第2次訓練模式，loss為:0.668 正確率為72.277，測試模式，loss為:0.717 正確率為70.303
第3次訓練模式，loss為:0.6 正確率為75.213，測試模式，loss為:0.754 正確率為68.611
第4次訓練模式，loss為:0.538 正確率為77.847，測試模式，loss為:0.802 正確率為67.778
第5次訓練模式，loss為:0.477 正確率為80.497，測試模式，loss為:0.842 正確率為68.986


預測

In [ ]:
#產生Model,進行預測
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('trained_model/config.json')
model = BertForSequenceClassification.from_pretrained('trained_model/pytorch_model.bin', from_tf=bool('.ckpt' in 'bert-base-uncased'), config=config)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# set device
device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def to_input_id(sentence_input):
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input)))

In [ ]:
#開始預測
eval_sents=[]
for index, row in test.iterrows():
  input_id=to_input_id(row['Phrase'])
  assert len(input_id) <= 512
  while len(input_id)<512:
    input_id.append(0)
  input_ids = torch.LongTensor(input_id).to(device).unsqueeze(0) 
  # predict時，因為沒有label所以沒有loss
  outputs = model(input_ids)
  predicts = outputs[:2]
  predicts = predicts[0]
  max_val = torch.max(predicts)
  predict_label = (predicts == max_val).cpu().nonzero().numpy()[0][1] # 在第1維度取最大值並返回索引值 
  eval_sents.append(predict_label)

In [ ]:
print(len(eval_sents))

66292


In [ ]:
submit.sample(5)

,PhraseId,Sentiment
56719,212780,2
52205,208266,2
28391,184452,2
43361,199422,2
31512,187573,2


In [ ]:
test.sample(5)

,PhraseId,Sentiment
63965,220026,2
4942,161003,2
694,156755,2
33150,189211,2
32063,188124,2


In [ ]:
test['Sentiment']=eval_sents
del test['SentenceId']
del test['Phrase']

In [ ]:
test.to_csv('result.csv',index=False)